hello world 👋 

In [18]:
import optuna
from catboost import CatBoostClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cr, train_test_split

In [22]:
X, y = fetch_openml(data_id=1597, data_home="openml_download", return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 29 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284807 non-null  float64
 1   V2      284807 non-null  float64
 2   V3      284807 non-null  float64
 3   V4      284807 non-null  float64
 4   V5      284807 non-null  float64
 5   V6      284807 non-null  float64
 6   V7      284807 non-null  float64
 7   V8      284807 non-null  float64
 8   V9      284807 non-null  float64
 9   V10     284807 non-null  float64
 10  V11     284807 non-null  float64
 11  V12     284807 non-null  float64
 12  V13     284807 non-null  float64
 13  V14     284807 non-null  float64
 14  V15     284807 non-null  float64
 15  V16     284807 non-null  float64
 16  V17     284807 non-null  float64
 17  V18     284807 non-null  float64
 18  V19     284807 non-null  float64
 19  V20     284807 non-null  float64
 20  V21     284807 non-null  float64
 21  V22     28

In [16]:
classifier = CatBoostClassifier()

In [17]:
classifier.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.121128
0:	learn: 0.3071897	test: 0.3072926	best: 0.3072926 (0)	total: 14.2ms	remaining: 14.2s
1:	learn: 0.1536840	test: 0.1535857	best: 0.1535857 (1)	total: 27.5ms	remaining: 13.7s
2:	learn: 0.0736323	test: 0.0736258	best: 0.0736258 (2)	total: 40ms	remaining: 13.3s
3:	learn: 0.0381991	test: 0.0381996	best: 0.0381996 (3)	total: 53.2ms	remaining: 13.3s
4:	learn: 0.0219160	test: 0.0219192	best: 0.0219192 (4)	total: 65.9ms	remaining: 13.1s
5:	learn: 0.0138181	test: 0.0138475	best: 0.0138475 (5)	total: 78.2ms	remaining: 13s
6:	learn: 0.0094805	test: 0.0095179	best: 0.0095179 (6)	total: 91ms	remaining: 12.9s
7:	learn: 0.0071271	test: 0.0071443	best: 0.0071443 (7)	total: 104ms	remaining: 12.9s
8:	learn: 0.0056565	test: 0.0056889	best: 0.0056889 (8)	total: 116ms	remaining: 12.8s
9:	learn: 0.0046348	test: 0.0046960	best: 0.0046960 (9)	total: 129ms	remaining: 12.7s
10:	learn: 0.0040311	test: 0.0041042	best: 0.0041042 (10)	total: 141ms	remaining: 12.7s
11:	learn: 0.0036231	

1. Tree parameters
    * Depth
    * min_data_in_leaf
    * grow_policy
2. Sampling parameters
    * Subsample
    * colsample_bylevel
    * sampling_frquency
3. Regularization parameters
    * penalties_coefficient
    * first_feature_use_penalties
    * leaf_estimation_backtracking
4. Learning rate
    * Iterations
    * learning_rate
    * model_shrink_rate
    * boost_from_average

- what are hyperparams, and why tune
- gridsearch, randomsearch, lhs search -> pro cons of each
- optuna in one go
- optuna in steps 
- compare time should be less, for same or better performance 